In [13]:
# tools를 사용한 함수모드
from openai import OpenAI
import datetime
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()


def get_current_datetime():
    "혅재 날짜와 시간을 반환하는 함수입니다."

    now = datetime.datetime.now()
    current = now.strftime("%Y-%m-%d %H:%M:%S")
    print("current:", current)
    return current


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "현재 날짜와 시간을 반환하는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {},  # 함수가 받을 수 있는 파라미터 정의
                "required": [],  # 필수 파라미터 정의
            },
        },
    }
]


def chat_with_tool(user_input):
    messages = [{"role": "user", "content": user_input}]

    # 도구를 사용하여 client에게 메시지를 보냅니다.
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    print("response_message:", response)
    # 도구 호출 정보를 가져옵니다.
    tool_calls = response_message.tool_calls

    # 도구 호출
    if tool_calls:
        tool_call = tool_calls[0]
        # 사용 가능한 함수 정의
        available_functions = {
            "get_current_datetime": get_current_datetime,
        }
        messages.append(response_message)
        # 도구 호출 함수 이름을 가져옵니다.
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]


        # 함수를 호출하고 응답을 반환합니다.
        function_response = function_to_call()
        messages.append({"role": "user", "content": function_response})
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            )
        return response.choices[0].message.content


user_query = "오늘이 몇일이지? 지금 몇시야?"
result = chat_with_tool(user_query)
print("tool chat: ", result)

response_message: ChatCompletion(id='chatcmpl-9jfRKaetC2a4rdyFFApDLNs4ZzaQo', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_C1BHwlHIVB14j5nJ0laNP52a', function=Function(arguments='{}', name='get_current_datetime'), type='function'), ChatCompletionMessageToolCall(id='call_fq3IKLG3BfNeT73JGU5V3r28', function=Function(arguments='{}', name='get_current_datetime'), type='function')]))], created=1720671082, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=40, prompt_tokens=65, total_tokens=105))
current: 2024-07-11 13:11:21


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_C1BHwlHIVB14j5nJ0laNP52a, call_fq3IKLG3BfNeT73JGU5V3r28", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

In [11]:
# tools를 사용한 함수모드
from openai import OpenAI
import datetime
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()


def get_current_datetime():
    "혅재 날짜와 시간을 반환하는 함수입니다."

    now = datetime.datetime.now()
    current = now.strftime("%Y-%m-%d %H:%M:%S")
    return current


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_datetime",
            "description": "현재 날짜와 시간을 반환하는 함수입니다.",
            "parameters": {
                "type": "object",
                "properties": {},  # 함수가 받을 수 있는 파라미터 정의
                "required": [],  # 필수 파라미터 정의
            },
        },
    }
]


def chat_with_tool(user_input):
    messages = [{"role": "user", "content": user_input}]
    # 도구를 사용하여 client에게 메시지를 보냅니다.
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    finished_reason = response.choices[0].finish_reason

    if finished_reason == "tool_calls":
        
        # 사용 가능한 함수 정의
        available_functions = {
            "get_current_datetime": get_current_datetime,
        }
        # 도구 호출 함수 이름을 가져옵니다.
        function_name = response.choices[0].message.tool_calls[0].function.name
        function_to_call = available_functions[function_name]


        # 함수를 호출하고 응답을 반환합니다.
        function_response = function_to_call()
        
        return function_response


user_query = "현재 시간이 몇시야?"
result = chat_with_tool(user_query)
print("tool chat: ", result)

tool chat:  2024-07-11 13:01:32
